# SECTION 1: Introduction

## SECTION 1.1: Corpora

Corpus contains raw text (ASCII/UTF-8) and metadata.

In [16]:
import nltk

In [17]:
from nltk.corpus import words

In [18]:
from nltk.corpus import reuters

In [19]:
from nltk.corpus import brown

In [20]:
brown.categories()

['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

## SECTION 1.2: Tokenization

2 types of words:
1. content words
2. stopwords

Pure Python, spaCy, or NLTK can be used.

In [21]:
import spacy

In [22]:
import en_core_web_sm

In [23]:
nlp = en_core_web_sm.load()
text = "Mary, don't slap the green witch"
print(
    [
        str(token) for token
        in nlp(text.lower())
    ]
)

['mary', ',', 'do', "n't", 'slap', 'the', 'green', 'witch']


In [24]:
from nltk.tokenize import \
TweetTokenizer

In [25]:
tweet = "Snow White and the Seven Degrees #MakeAMovieCold @midnight :-)"
tokenizer = TweetTokenizer()
print(
    tokenizer.tokenize(tweet.lower())
)

['snow', 'white', 'and', 'the', 'seven', 'degrees', '#makeamoviecold', '@midnight', ':-)']


NLTK tweet tokenizer preserves hashtags, handles, and smiles.

## SECTION 1.3: WordNet

WordNet is a large lexical database in English.

In [26]:
from nltk.corpus import wordnet

In [29]:
from nltk.corpus import wordnet

synonyms = []
antonyms = []

for syn in wordnet.synsets('good'):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(
                l.antonyms()[0].name()
            )

print(set(synonyms))
print(set(antonyms))

{'secure', 'unspoiled', 'skilful', 'thoroughly', 'right', 'unspoilt', 'practiced', 'honest', 'upright', 'commodity', 'goodness', 'adept', 'estimable', 'effective', 'beneficial', 'trade_good', 'good', 'expert', 'full', 'safe', 'salutary', 'honorable', 'just', 'soundly', 'in_force', 'dear', 'well', 'ripe', 'serious', 'sound', 'proficient', 'undecomposed', 'near', 'skillful', 'respectable', 'dependable', 'in_effect'}
{'evil', 'badness', 'ill', 'evilness', 'bad'}


## SECTION 1.4: Grammartical Analysis

In [30]:
doc = nlp(u"Apple is looking at buying U.K. startup for $1 billion")
for token in doc:
    print(token.text, token.pos_, token.dep_)

Apple PROPN nsubj
is AUX aux
looking VERB ROOT
at ADP prep
buying VERB pcomp
U.K. PROPN dobj
startup NOUN dep
for ADP prep
$ SYM quantmod
1 NUM compound
billion NUM pobj


## SECTION 1.5: Dependency Parsing

In [31]:
from spacy import displacy

In [32]:
from spacy import displacy

displacy.render(doc, 
                style='dep', 
                jupyter='True', 
                options={'distance':90})

## SECTION 1.6: Named Entity Recognition (NER)

In [33]:
doc = nlp(
    'I just bought 2 shares at 9 am because the stock went up 30% in just 2 days according to the WSJ'
)
displacy.render(doc,
              style='ent',
              jupyter='True')

# SECTION 2: Text Representation

**Why is representation important?**

Text representation scheme must facilitate the extraction of the features.

The *semantics* (meaning) of a sentence comes from the 4 steps:
1. Break the sentence into lexical units
2. Derive the meaning of each unit
3. Understand the syntactic (grammatical) structure of the sentence
4. Understand the context in which the sentence appears

**What is text representation?**

Text representation is the conversion from of raw text into a suitable numerical form.

**Legacy Techniques**
1. one-hot encoding
2. bag of words
3. n-gram
4. TF-IDF

## SECTION 2.1: One-hot Encoding

1. No information about words relations
2. Must pre-determine vocabulary size
3. Size of input vector scales with size of vocabulary
4. "Out-of-vocabulary" (OOV) problem

In [34]:
import numpy as np

In [45]:
def one_hot(word, word_dict):
    vector = np.zeros(len(word_dict))
    # vector[word_dict[word]] = 1
    if word in word_dict:
        vector[word_dict[word]] = 1
    
    return vector

words = ['rome', 'paris', 'italy', 'france']
word_dict = {word: idx for idx, word in enumerate(words)}

print(one_hot("paris", word_dict))

[0. 1. 0. 0.]


## SECTION 2.2: Bag of Words

Bag of words is a vector representation of a text produced by simply adding up all the one-hot encoded vectors:

1. Vectors simply contain the number of times each word appears in our document.
2. *Orderless*
3. No notion of similarity

In [48]:
vocabulary_size = 50

text_words = [
    'rome', 'paris', 'italy', 'france',
    'rome', 'magnificent', 'tourism', 'night',
    'tourism', 'tourism'
]

# bow = np.zeros(vocabulary_size)
bow = [0] * len(word_dict)

for word in text_words:
    hot_word = one_hot(word, word_dict)
    # bow += hot_word
    bow = [sum(x) for x in zip(bow, hot_word)]  # Element-wise sum

print(bow)

bow[word_dict["paris"]]

[2.0, 1.0, 1.0, 1.0]


1.0

## SECTION 2.3: N-gram Model

N-gram model is a contiguous sequence of n items from a given sample of text.

1. Vocabulary = set of all n-grams in corpus
2. No notion of similarity

In [49]:
from nltk import ngrams

text = "Machines take me by surprise with great frequency."

n = 5
pentagrams = ngrams(nltk.word_tokenize(text), n)

for grams in pentagrams:
    print(grams)

('Machines', 'take', 'me', 'by', 'surprise')
('take', 'me', 'by', 'surprise', 'with')
('me', 'by', 'surprise', 'with', 'great')
('by', 'surprise', 'with', 'great', 'frequency')
('surprise', 'with', 'great', 'frequency', '.')


## SECTION 2.4: Collocations

A collocation is a sequence of words that occur together unusually often.

`nltk.collocations` can help identifying phrases that act like single words.

In the example below, bi-grams are paired with a "more likely to occur" score:

In [51]:
import nltk.collocations
import nltk.corpus
import collections

bgm = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(
    nltk.corpus.brown.words()
)
scored = finder.score_ngrams(bgm.likelihood_ratio)

# Group bigrams by first word in bigram
prefix_keys = collections.defaultdict(list)
for key, scores in scored:
    prefix_keys[key[0]].append((key[1], scores))

# Sorted key bigrams by strongest association
for key in prefix_keys:
    prefix_keys[key].sort(key = lambda x: -x[1])

print('New', prefix_keys['New'][:5])

New [('York', 4634.968955894195), ('Orleans', 611.6951040864856), ('England', 557.5789255397682), ('Jersey', 265.2781409189113), ('Testament', 182.6595658588261)]


## SECTION 2.5: Term Frequency

In [53]:
import gzip

from collections import Counter

In [55]:
data = []

'''
for line in gzip.open("F:/EDGE/LearningAI/sample_text.txt", 'rt'):
    data.extend(line.strip().split())
'''

for line in open("F:/EDGE/LearningAI/sample_text.txt", 'rt'):
    data.extend(line.strip().split())

'''
# Open the file and read its contents
with open(file_path, 'r') as file:
    # Read the file and split the text into words
    words = file.read().split()
'''

counts = Counter(data)

sorted_counts = sorted(list(counts.items()), key=lambda x:x[1], 
                       reverse=True)

for word, count in sorted_counts[:5]:
    print(word, count)

the 26
a 14
and 10
their 7
of 6


In [56]:
import nltk
all_words = nltk.FreqDist(data)
print(all_words.most_common(5))

[('the', 26), ('a', 14), ('and', 10), ('their', 7), ('of', 6)]


## SECTION 2.6: 3 Ways to Remove Stopwords

**(1/3) Remove the most common 100 words:**

In [57]:
stopwords = set(
    [
        word for word,
        count in sorted_counts[:100]
    ]
)

clean_data = []

for word in data:
    if word not in stopwords:
        clean_data.append(word)

**(2/3) Use nltk predefined stopwords:**

In [58]:
from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('english')

print('Number of stopwords: %d' % len(nltk_stopwords))
print('First five stop words: %s' % list(nltk_stopwords)[:5])

Number of stopwords: 179
First five stop words: ['i', 'me', 'my', 'myself', 'we']


**(3/3) Use spaCy predefined stopwords**

In [59]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
print('Number of stopwords: %d' % len(spacy_stopwords))
print('First five stop words: %s' % list(spacy_stopwords)[:5])

Number of stopwords: 326
First five stop words: ['thru', 'very', '‘ll', 'them', '‘s']


In [64]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Combine the list of words into a single string
data_str = ' '.join(data)

# Process your text using spaCy
doc = nlp(data_str)

# Extract tokens, excluding stop words
tokens = [token.text for token in doc 
          if not nlp.vocab[token.text].is_stop]

print(tokens)

['GENERATED', 'CHATGPT', 'time', ',', 'remote', 'island', 'nestled', 'heart', 'Pacific', ',', 'adventurer', 'named', 'Alex', 'found', 'stranded', 'unexpected', 'shipwreck', '.', 'island', ',', 'lush', 'vibrant', 'foliage', 'adorned', 'pristine', 'beaches', ',', 'like', 'paradise', 'glance', '.', ',', 'Alex', 'soon', 'realized', 'beauty', 'concealed', 'challenges', 'lay', 'ahead', '.', 'days', ',', 'Alex', 'scoured', 'shoreline', 'salvageable', 'items', 'shipwreck', '.', 'debris', ',', 'found', 'crates', 'canned', 'food', ',', 'tattered', 'functional', 'tent', ',', 'waterproof', 'box', 'containing', 'matches', '.', 'resources', ',', 'established', 'small', 'camp', 'near', 'freshwater', 'stream', ',', 'ensuring', 'basic', 'needs', 'met', '.', 'Days', 'turned', 'weeks', ',', 'Alex', 'survival', 'instincts', 'kicked', '.', 'began', 'exploring', 'island', 'interior', ',', 'learning', 'identify', 'edible', 'plants', 'honing', 'fishing', 'skills', '.', 'way', ',', 'encountered', 'peculiar', '

## SECTION 2.7: TF-IDF

TF-IDF reflects how important a word is to a document in a corpus.

$$W_{x,y}=tf_{x,y}\times log(\frac{N}{df_{x}})$$

Here,
> $W_{x,y}$ is the TF-IDF, i.e. the term $x$ within the document $y$

> $tf_{x,y}$ is the frequency of $x$ in $y$

> $df_{x}$ is the number of documents containing $x$

> $N$ is the total number of documents

TF-IDF of common words is zero.

**(1/3) TF-IDF from Scratch:**

In [65]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bowCount)
    return tfDict

def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
    
    return idfDict

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

**(2/3) TF-IDF using Sklearn**

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer

D1 = "The car is driven on the road."
D2 = "The truck is driven on the highway."

vectorizer = TfidfVectorizer()
response = vectorizer.fit_transform([D1, D2])

print(response)

  (0, 5)	0.42471718586982765
  (0, 4)	0.30218977576862155
  (0, 1)	0.30218977576862155
  (0, 3)	0.30218977576862155
  (0, 0)	0.42471718586982765
  (0, 6)	0.6043795515372431
  (1, 2)	0.42471718586982765
  (1, 7)	0.42471718586982765
  (1, 4)	0.30218977576862155
  (1, 1)	0.30218977576862155
  (1, 3)	0.30218977576862155
  (1, 6)	0.6043795515372431


In [75]:
pprint(list(enumerate(vectorizer.get_feature_names())))

Pretty printing has been turned OFF


`fit` learns vocabulary and idf from the training set.

`transform` transforms documents to document-term matrix.

**(3/3) TF_IDF using Sklearn + StopWords**

In [78]:
from pprint import pprint

In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer

D1 = "The car is driven on the road."
D2 = "The truck is driven on the highway."

vectorizer = TfidfVectorizer(stop_words='english')
response = vectorizer.fit_transform([D1, D2])

print(response)

# Print the feature names
# feature_names = vectorizer.get_feature_names()
feature_names = vectorizer.get_feature_names_out()
pprint(list(enumerate(feature_names)))
'''
pprint(
    list(
        enumerate(
            vectorizer.get_feature_names()
        )
    )
)
'''

  (0, 3)	0.6316672017376245
  (0, 1)	0.4494364165239821
  (0, 0)	0.6316672017376245
  (1, 2)	0.6316672017376245
  (1, 4)	0.6316672017376245
  (1, 1)	0.4494364165239821
[(0, 'car'), (1, 'driven'), (2, 'highway'), (3, 'road'), (4, 'truck')]


'\npprint(\n    list(\n        enumerate(\n            vectorizer.get_feature_names()\n        )\n    )\n)\n'

# SECTION 3: Vector Models